## Create XG DB diabetes


In [1]:
import xgboost as xgb
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split


from giza.datasets import DatasetsLoader
from giza.agents import GizaAgent
from giza.zkcook import serialize_model

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


c:\Users\sathy\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = load_diabetes()
X, y = data.data, data.target

In [3]:
data

{'data': array([[ 0.03807591,  0.05068012,  0.06169621, ..., -0.00259226,
          0.01990749, -0.01764613],
        [-0.00188202, -0.04464164, -0.05147406, ..., -0.03949338,
         -0.06833155, -0.09220405],
        [ 0.08529891,  0.05068012,  0.04445121, ..., -0.00259226,
          0.00286131, -0.02593034],
        ...,
        [ 0.04170844,  0.05068012, -0.01590626, ..., -0.01107952,
         -0.04688253,  0.01549073],
        [-0.04547248, -0.04464164,  0.03906215, ...,  0.02655962,
          0.04452873, -0.02593034],
        [-0.04547248, -0.04464164, -0.0730303 , ..., -0.03949338,
         -0.00422151,  0.00306441]]),
 'target': array([151.,  75., 141., 206., 135.,  97., 138.,  63., 110., 310., 101.,
         69., 179., 185., 118., 171., 166., 144.,  97., 168.,  68.,  49.,
         68., 245., 184., 202., 137.,  85., 131., 283., 129.,  59., 341.,
         87.,  65., 102., 265., 276., 252.,  90., 100.,  55.,  61.,  92.,
        259.,  53., 190., 142.,  75., 142., 155., 225.,  59

In [9]:
#X, y = data.data, data.target

# Drop rows with missing values
df_cleaned = data.dropna(subset=['bmi'])

# Prepare your data
X = df_cleaned[['bmi']]
y = df_cleaned['expenses']


In [10]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a linear regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

LinearRegression()

In [11]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# Define the initial types for the ONNX model
#initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]  ]))]

initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]  ]))]

# Convert the scikit-learn model to ONNX
onnx_model = convert_sklearn(model, initial_types=initial_type)

# Save the ONNX model to a file
with open("linear_regression-ins.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## Save the model
### Save the model in Json format

## Transpile your model to Orion Cairo
### We will use Giza-CLI to transpile our saved model to Orion Cairo.


$ giza transpile 

## Deploy an inference endpoint
### Now that our model is transpiled to Cairo we can deploy an endpoint to run verifiable inferences. We will use Giza CLI again to run and deploy an endpoint. Ensure to replace model-id and version-id with your ids provided during transpilation.


giza endpoints deploy --model-id 665 --version-id 1

$ giza endpoints deploy --model-id XXX --version-id X


## Run a verifiable inference
##### To streamline verifiable inference, you might consider using the endpoint URL obtained after transpilation. However, this approach requires manual serialization of the input for the Cairo program and handling the deserialization process. To make this process more user-friendly and keep you within a Python environment, we've introduced a Python SDK designed to facilitate the creation of ML workflows and execution of verifiable predictions. When you initiate a prediction, our system automatically retrieves the endpoint URL you deployed earlier, converts your input into Cairo-compatible format, executes the prediction, and then converts the output back into a numpy object. 

In [13]:
from giza.agents.model import GizaModel

MODEL_ID = 719  # Update with your model ID
VERSION_ID = 3  # Update with your version ID

def prediction(input, model_id, version_id):
    model = GizaModel(id=model_id, version=version_id)

    (result, proof_id) = model.predict(
        input_feed={'input': input}, verifiable=True
    )

    return result, proof_id

def execution():
    # The input data type should match the model's expected input

    input = np.array([[30.1]]).astype(np.float32)
    #input = np.array([[31,28.1]]).astype(np.float32)
    (result, proof_id) = prediction(input, MODEL_ID, VERSION_ID)

    print(
        f"Predicted value for input {input.flatten()[0]} is {result[0].flatten()[0]}")

    return result, proof_id


execution()

🚀 Starting deserialization process...
✅ Deserialization completed! 🎉
Predicted value for input 30.100000381469727 is 13053.928756713867


(array([[13053.92875671]]), 'b7ee4fb8426644c3a6f12a7a72a95b8d')

## Download the proof
#### Initiating a verifiable inference sets off a proving job on our server, sparing you the complexities of installing and configuring the prover yourself. Upon completion, you can download your proof.

First, let's check the status of the proving job to ensure that it has been completed.

$ giza endpoints get-proof --endpoint-id XXX --proof-id "xxxxxxxxxxxxxxxxxxxxxxxxxx"
[giza][2024-05-30 00:40:39.691] Getting proof from endpoint xxx
{
  "id": 967,
  "job_id": 1121,
  "metrics": {
    "proving_time": 17.249508
  },
  "created_date": "2024-05-30T07:33:12.532659"
}

Once the proof is ready, you can download it.

$ giza endpoints download-proof --endpoint-id xxx --proof-id "xxxxxxxxxxxxxxxxx" --output-path zk_xgboost.proof
[giza][2024-05-30 00:51:52.048] Getting proof from endpoint xxx ✅
[giza][2024-05-30 00:51:53.800] Proof downloaded to zk_xgboost.proof ✅
(giza3)

## Verify the proof
#### Finally, you can verify the proof.

$ giza verify --proof-id 967
[giza][2024-05-30 00:56:05.847] Verifying proof...
[giza][2024-05-30 00:56:07.140] Verification result: True
[giza][2024-05-30 00:56:07.145] Verification time: 0.454667226
(giza3)